In [1]:
import sqlite3
import pandas as pd
import plotly.express as px
import numpy as np
import sqlalchemy as sa
from src.models import *

from src.pkmn_constants import *

### Get Data

In [2]:
with Session(engine) as session:

    # current playthrough
    playthrough = (
        session
        .query(Playthrough)
        .order_by(Playthrough.adventure_started.desc())
        .first()
    )
    print(f"Current playthrough: {playthrough}")

    # get event data
    query = (
        session
        .query(Playthrough,Event,TeamMemberEntry)
        .filter(Playthrough.adventure_started == (
            session
            .query(sa.func.max(Playthrough.adventure_started))
            .scalar_subquery()
        ))
        .join(Playthrough.events, isouter=True)
        .join(Event.team_member_entries, isouter=True)
    )
    data = pd.read_sql(query.statement, engine)

    # get pokemon color
    team_member_entries = (
        session
        .query(TeamMemberEntry,TeamMember)
        .filter(TeamMember.playthrough == playthrough)
        .join(TeamMemberEntry.team_member)
        .order_by(TeamMemberEntry.no)
    ).all()
    team_member_color_discrete_map = {}
    team_member_slots = []
    team = set()
    for item in team_member_entries:
        team_member_entry, team_member = item.tuple()

        # get team_member_name
        data.loc[data['team_member_slot'] == team_member.slot, 'team_member'] = team_member.to_str(session)

        # color
        if team_member_entry.type1:
            team_member_color_discrete_map[team_member.to_str(session)] = TYPES[team_member_entry.type1]

        # make team
        team.add(team_member.to_str(session))
data = data.sort_values(by=['no'])
data

Current playthrough: Black 2023-06-24


,id_no,name,version,adventure_started,no,playthrough_id_no,location_name,location_region,event_type,event_name,...,no_1,team_member_playthrough_id_no,team_member_slot,event_no,level,species,dex_no,type1,type2,team_member
139,26852,Ben,Black,2023-06-24,1,26852,Nuvema Town,Unova,Gift,Lillipup,...,1.0,26852,1.0,1.0,5.0,Lillipup,506.0,Normal,None,Stoutland
137,26852,Ben,Black,2023-06-24,2,26852,Nuvema Town,Unova,Battle,PKMN Trainer Bianca (Single),...,NaN,None,NaN,NaN,NaN,None,NaN,None,None,NaN
138,26852,Ben,Black,2023-06-24,3,26852,Nuvema Town,Unova,Battle,PKMN Trainer Cheren (Single),...,NaN,None,NaN,NaN,NaN,None,NaN,None,None,NaN
0,26852,Ben,Black,2023-06-24,4,26852,Accumula Town,Unova,Battle,PKMN Trainer N (Single),...,2.0,26852,1.0,4.0,7.0,None,NaN,None,None,Stoutland
272,26852,Ben,Black,2023-06-24,5,26852,Route 2,Unova,Battle,Youngster Jimmy (Single),...,3.0,26852,1.0,5.0,8.0,None,NaN,None,None,Stoutland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,26852,Ben,Black,2023-06-24,332,26852,Pokémon League,Unova,Battle,Elite Four Caitlin (Single),...,208.0,26852,2.0,332.0,54.0,None,NaN,None,None,Simisear
184,26852,Ben,Black,2023-06-24,332,26852,Pokémon League,Unova,Battle,Elite Four Caitlin (Single),...,207.0,26852,2.0,332.0,53.0,None,NaN,None,None,Simisear
227,26852,Ben,Black,2023-06-24,333,26852,Pokémon League,Unova,Battle,Team Plasma N (Single),...,209.0,26852,1.0,333.0,51.0,None,NaN,None,None,Stoutland
229,26852,Ben,Black,2023-06-24,334,26852,Pokémon League,Unova,Battle,Team Plasma N (Single),...,211.0,26852,5.0,334.0,54.0,None,NaN,None,None,Mienshao


## Level Data

### Team Member Levels over Time

In [3]:
data['no'].max()

334

In [4]:
max_event_no = data['no'].max()
team_levels = (
    data
    .pivot_table(values='level',index=['no','location_name'],columns='team_member',aggfunc='max')
    .reset_index()
    .set_index('no')
    .reindex(np.arange(data.index.min(),max_event_no+1))
    .rename(columns={'location_name':'_location_name'})
)
team_levels[list(team)] = team_levels[list(team)].ffill()
display(team_levels)
(px
    .line(team_levels,
        x=team_levels.index,
        y=list(team),
        color_discrete_map=team_member_color_discrete_map,
        title=f"Team Member Levels")
    .update_layout(hovermode='x')
    .update_traces(hovertemplate="%{y}")
)

team_member,_location_name,Archeops,Mienshao,Pawniard,Seismitoad,Simisear,Stoutland
no,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nuvema Town,NaN,NaN,NaN,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,NaN,NaN,NaN,NaN,NaN,NaN,5.0
4,Accumula Town,NaN,NaN,NaN,NaN,NaN,7.0
...,...,...,...,...,...,...,...
330,Pokémon League,53.0,53.0,51.0,51.0,52.0,50.0
331,NaN,53.0,53.0,51.0,51.0,52.0,50.0
332,Pokémon League,53.0,53.0,51.0,51.0,54.0,50.0


### Starting Level at Location

In [5]:
team_levels['_max'] = team_levels[list(team)].T.max()
_vals = team_levels.groupby('_location_name')['_max'].max().sort_values()
px.bar(x=_vals.index,y=_vals.values,title="Max Level by Location")